# Lineare Regression
In diesem Notebook realisieren wir einige Beispiele für die Lineare Regression. Wir beginnen mit einem sehr einfachen Beispiel. Wir erstellen ein Pandas-Dataframe mit x- und y-Werten:

In [ ]:
import pandas as pd

df = pd.DataFrame({"x":[25, 27, 29, 35, 40, 45, 46, 50, 55, 58],
                  "y":[127, 125, 132, 140, 145, 152, 160, 165, 170, 175]})
df



Wir plotten die Daten (Scatterplot) mit Hilfe von Matplotlib:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.scatter(df.y, df.y)
plt.show()

Nun erstellen wir ein Modell für die Lineare Regression mit Hilfe der Funktion *ols* aus dem Package *statsmodells*. Es gibt auch viele andere Möglichkeiten, dies in Python zu realisieren, aber *statsmodels* ist sehr mächtig, insbesondere was die Verwendung von kategorialen Variablen angeht (dazu später ein Beispiel). Wir geben die Koeffizienten aus, anschließend weitere Details zum Modell. Mit *summary* geben wir die Infos zum Modell aus, insbesondere die Koeffizienten $\beta_0$ und $\beta_1$. Da wir nur 10 Beobachtungen haben, wird noch eine Warnung ausgegeben.

In [ ]:
import statsmodels.formula.api as smf 

model = smf.ols("y~x", data=df).fit()
model.summary()

Eine wichtige Kennzahl für die Qualität eines Regressionsmodells ist das sog. **Bestimmtheitsmaß** $R^2$. Die Tabelle oben gibt hierfür einen Wert von 0,984 an. Dieser kann auch manuell mit folgender Formel berechnet werden:

$R^2=1-\frac{\sum (y_{i}-\hat{y})^{2}}{\sum (y_{i}-\bar{y})^{2}}$

Das gleiche Ergebnis erhalten wir mit:

In [ ]:
1 - (((df.y-model.predict(df.x))**2).sum() / ((df.y-df.y.mean())**2).sum() )

In [ ]:
from scipy.stats import pearsonr
corr,__ = pearsonr(df.x, df.y)
print(corr)

Wir erstellen nochmal einen Scatterplot mit den Daten, zusätzlich fügen wir noch die Regressinsgerade hinzu:

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(df.x, df.y)
plt.plot(df.x, model.predict(df.x))
plt.show()

## Beispiel: Speiseeis-Umsatz
Wir laden einen Datensatz mit Umsatzzahlen (in Tsd. EUR) einer Supermarktkette in Abhängigkeit von der Außentemperatur:

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/troescherw/datasets/master/speiseeis_umsatz.csv"
umsatz = pd.read_csv(url)
umsatz.head()

Wir erstellen wieder einen Scatterplot:

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(umsatz.Temperatur, umsatz.Umsatz)

Wir erstellen ein Regressionsmodell und geben Infos zum Modell aus:

In [ ]:
import statsmodels.formula.api as smf
model = smf.ols("Umsatz~Temperatur", data=umsatz).fit()
model.summary()

## Noch ein Beispiel: Bierpreis auf dem Oktoberfest
Wir laden reale Daten (Quelle: Open Data Portal der Stadt München) bzgl. der Bierpreisentwicklung auf dem Oktoberfest:

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/troescherw/datasets/master/oktoberfestbierpreis19852018.csv"
bierpreise = pd.read_csv(url)
bierpreise.head()

Wir erstellen einen Scatterplot:

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(bierpreise.jahr, bierpreise.bier_preis)
plt.xlabel("Jahr")
plt.ylabel("Preis pro Maß")
plt.title("Entwicklung der Bierpreise auf dem Münchner Oktoberfest")
plt.show()

Offensichtlich ein erstaunlich linearer Zusammenhang! Wir erstellen ein Modell:

In [ ]:
import statsmodels.formula.api as smf
model = ols("bier_preis~jahr", data=bierpreise).fit()
model.summary()

Das Modell verfügt über ein Bestimmtheitsmaß $R^2$ von 0,99! Die Koeffizienten zeigen, dass pro Jahr die Maß Bier im Mittel um ca. 25 Cent teuerer wurde!

## Berechnung der Koeffizienten mit Hilfe der Linearen Algebra
Wir erstellen eine Funktion, die die Koeffizienten mit Hilfe der Linearen Algebra berechnet (Koeffizientenbestimmung eines überbestimmten Gleichungssystems mit Hilfe der Moore-Penrose-Pseudoinversen). Wir verwenden als Beispiel unsere Bierpreis-Daten und geben zum Vergleich nochmal die Koeffizienten unseres Modells aus, die die *ols*-Funktion berechnet hat:

In [ ]:
import numpy as np

# Funktion erstellen
def koef(X, y):
    X = pd.DataFrame(X)
    X.insert(0, "Beta_0", 1)
    return np.linalg.inv(X.T @ X) @ X.T @ y

beta_0, beta_1 = koef(bierpreise.jahr, bierpreise.bier_preis)
print(beta_0, beta_1)

## Nicht-lineare Daten
Bei den bisher verwendeten Datensätzen bestand ein guter linearer Zusammenhang zwischen der unabhängigen Variable (y) und der unabhängigen Variable (X). Was aber tun, wenn zum Beispiel ein logarithmischer Zusammenhang besteht?

Als Beispiel betrachten wir das Moorsche Gesetz, das besagt, dass sich alle 2 Jahre die Anzahl der Transistoren auf einer bestimmten Fläche verdoppeln. Der folgende Datensatz, der die Anzahl der Transistoren seit dem Jahr 1970 beinhaltet, zeigt, dass das Moorsche Gesetzt (immer noch) stimmt:

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/troescherw/datasets/master/transistoren.csv"
df = pd.read_csv(url, delimiter=";")
df.head()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(df.Year_after_1970, df.Transistor_count)
plt.show()


Tatsächlich sieht das nach expontiellem Wachstum aus! Wir wollen dies nun in eine lineare Funktion überführen. Dazu logarithmieren wir die Daten für die Anzahl der Transistoren und fügen diese Daten dem Datensatz hinzu:

In [ ]:
#df.insert(0,"Transistor_count_log", df.Transistor_count)
df["Transistor_count_log"] = np.log(df.Transistor_count)

# Scatterplot
import matplotlib.pyplot as plt
plt.scatter(df.Year_after_1970, df.Transistor_count_log)
plt.xlabel("Jahr nach 1970")
plt.ylabel("Anzahl Transistoren (log.)")
plt.title("Moorsches Gesetz")
plt.show()


Das sieht schon wesentlich "linearer" aus! Nun können wir auch eine Lineare Regression durchführen:

In [ ]:
import statsmodels.formula.api as smf
model = smf.ols("Transistor_count_log ~ Year_after_1970", data=df).fit()
model.summary()

Pro Jahr wächst also die Anzahl Prozessoren um den Faktor $2^{0,35}$, was ungefähr 2, also einer Verdopplung, entspricht.

 ## Multiple Regression mit Dummy-Variablen
 Bisher hatten wir nur eine unabhängige Variable. Im folgenden Beispiel wollen wir ein Vorhersagemodell für den Mietpreis von Wohnungen in Abhängigkeit von
 
 - der Größe der Wohnung in qm
 - der Lage (Innenstadt, Außenbezirk oder Umland)
 
erstellen. Da es sich bei der Lage um eine kategoriale Variable handelt, müssen wir dieses Feature Dummy-kodieren (One-Hot-Encoding). Praktischerweise übernimmt das für uns die Funktion *C* (für "categorical") aus *statsmodels*. Eine andere Möglichkeit wäre, die Funktion *get_dummies* aus Pandas. Dieser Funktion sollte man auch noch das Argument *drop_first=True* übergeben, damit wir nicht in die "Dummy-Variable-Trap" laufen! Die *C*-Funktion übernimmt das praktischerweise automatisch!

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

url = "https://raw.githubusercontent.com/troescherw/datasets/master/wohnungen.csv"
df = pd.read_csv(url, delimiter=";")
df.head()

In [ ]:
import statsmodels.formula.api as smf

model = smf.ols("Mietpreis~Quadratmeter+C(Lage)", data=df).fit()
model.summary()

Wir erkennen an den Koeffizienten, dass nur die Lagen *Innenstadt* und *Umland* vorhanden sind. Die Kategorie *Aussenbezirk* wurde also automatisch entfernt und ist somit in $\beta_0$ eingeflossen.

Somit ergeben sich folgende Erkenntnisse:

- Mit jedem Quadratmeter mehr steigt der Mietpreis einer Wohnung in der Lage "Aussenbezirk" um ca. 17,50 Euro.
- Eine Wohnung im Umland ist gegenüber einer Wohnung im Aussenbezirk im Schnitt um 152 Euro günstiger.
- Eine Wohnung mit Innenstadtlage ist hingegen im Schnitt um ca. 653 Euro teurer als eine Wohnung im Aussenbezirk.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

qm = np.arange(df.Quadratmeter.min(), df.Quadratmeter.max())
mieten_aussen = pd.DataFrame({"Quadratmeter":qm, "Lage":"Aussenbezirk"})
mieten_umland = pd.DataFrame({"Quadratmeter":qm, "Lage":"Umland"})
mieten_innen  = pd.DataFrame({"Quadratmeter":qm, "Lage":"Innenstadt"})

plt.plot(qm, model.predict(mieten_aussen), label="Außenbezirk")
plt.plot(qm, model.predict(mieten_umland), label="Umland")
plt.plot(qm, model.predict(mieten_innen), label="Innenstadt")
plt.legend()
plt.xlabel("Quadratmeter")
plt.ylabel("Mietpreis")
plt.title("Wohnungspreise in Abhängigkeit von Fläche und Lage")
plt.show()